In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt

## Define fixed bed model 

In [2]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [3]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y):
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}}
    return dv_dict_overall

In [19]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#initialize_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv'
#initialize_point = '2021oct8_f313_b313_y15_5e6.csv'
initialize_point = '2021oct8_f293_b293_y50_5e6.csv'
#initialize_point = './priors/20210916_feed313_bath293.csv'
#init_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/fixed_bed_initial/T353y20_5e3.csv'
#initialize_point = './priors/20210916_feed313_bath293_5e3.csv'
#init_point = 'init_test.csv'
#init_point = 'T353y20_5e3.csv'

In [20]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=293.15, y=0.4, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [6]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}
#dv_pass = {'temp_feed': None, 'temp_bath':None , 'yfeed': None}
dv_pass = {'temp_feed': None, 'yfeed': None}

measurement = ['FCO2', 'temp']
measurement_extra = {'FCO2': [19], 'temp': [10, 19]}

t_measure = exp_time_
measurement_pass = {'FCO2': t_measure, 'temp': t_measure}

In [7]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


In [8]:
fim_init = [[0.0029331 , 0.01236752],
 [0.01236752 , 0.71783208]]

#f2 = open('jac_initial', 'rb')
#jac_init = pickle.load(f2)
#f2.close()

#jac_init = jac_init_1.copy()
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

[[0.0541581  0.        ]
 [0.22835955 0.8158946 ]]


In [ ]:
import os

scores = {} # scores is an empty dict already
target='jac_initial'
if os.path.getsize(target) > 0:      
    with open(target, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()
        
print(scores)

## Compute FIM

In [9]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
#sensi_opt = 'sequential_finite'
sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# feed, bath, y
exp1 = generate_exp(293.15, 293.15, 0.5)

In [10]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output='store_output',
                                formula='central')

if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else: 
    result.calculate_FIM(doe_object.jac, doe_object.jac_3D, doe_object.design_values)

The flattend measurements include: ['FCO219', 'temp10', 'temp19']
Sensitivity information is scaled by its corresponding parameter nominal value.
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is 2021oct8_f313_b313_y15_5e6.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  293.15
yfeed is fixed at  0.5
perturb names are: ['fitted_transport_coefficient_pert', 'ua_pert']
Perturbation parameters are set:
fitted_transport_coefficient_pert :  209.88
ua_pert :  15.424948470398375


KeyboardInterrupt: 

In [ ]:
#print(doe_object.jac)
print((doe_object.jac_3D))
print(np.shape(doe_object.jac_3D))

In [ ]:
result.calculate_FIM(doe_object.jac, doe_object.jac_3D, doe_object.design_values, jaco_involved_name=['FCO2'], jaco_involved_extra_index=[[19]])




In [ ]:
dd = 'fco2_10'

kt = dd.index('_')
print(dd.split('_')[1])
print(kt)

In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
print(doe_object.dsdp)

In [ ]:
import pickle

with open('store_output_botht', 'rb') as f:
    data = pickle.load(f)
    f.close()
print(data)

In [ ]:
doe_object.models[0]

In [ ]:
breakthrough_modify2(doe_object.models[2])

In [ ]:
def breakthrough_modify2(m, file=None, source="lab"):
    '''
    Draw breakthrough curve of the Pyomo model
    
    Arguments:
        m: Pyomo model
        file: when source = 'computer', this is where the computer experimental data is stored. Otherwise, its default is None
        source: if computer, it is plotting the computer experiment data. If lab, its comparing the experiments from the lab
    
    Return: None 
    
    Other: plot
    
    '''
    
    
    # unit: mol/m3
    outlet_den, _, _, _ = extract3d(m, m.C, 'CO2')
    
    # unit: cm/s
    outlet_vel, _, _, _ = extract2d(m, m.v)
    
    outlet_n2, _, _, _ = extract3d(m, m.C, 'N2')
    
    model_temp, _,_,_ = extract2d(m,m.temp)
    
    
    T = []
    for j in m.t:
        T.append(value(j))
        t_final = T[-1]
        
    
    if source == "lab":
        break_wvu = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/breakthrough_wvu.csv')
    
        exp = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/co2_breakthrough.csv')
        
        data_c1 = exp['FCO2']
        data_c = np.asarray(data_c1)
        data_t1 = exp['time']  
        data_t = np.asarray(data_t1*60+10)
        new_data = np.interp(T, data_t, data_c)
    
        fco2 = []
        for t in m.t:
            fco2.append(value(m.FCO2[0,19,t]))
        
        yco2 = []
        for i in range(len(fco2)):
            yco2.append(fco2[i]/fco2[-1])
            
        # real value of FCO2. for verification
        comp = outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1])
    
        plt.plot(exp['time']*60, exp['FCO2'],'b.', color='r', label='Experimental data')
        #plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.plot(T, yco2, label='Model prediction')
        
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
    elif source == "computer": 
        
        sol = pd.read_csv(file)
    
        #for i in range(len(time_exp)):    
        #    yco2_exp[time_exp[i]] = sol['FCO2'][i]
        #    temp_mid_exp[time_exp[i]] = sol['temp_mid'][i]
        #    temp_end_exp[time_exp[i]] = sol['temp_end'][i]
        
        exp_fco2 = np.asarray(sol['FCO2'].values.tolist())
        exp_temp_mid = np.asarray(sol['temp_mid'].values.tolist())
        exp_temp_end = np.asarray(sol['temp_end'].values.tolist())
        
        plt.plot(T, exp_fco2/exp_fco2[-1], label = 'Experimental data')
        plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
        
        plt.plot(T, exp_temp_mid, label='Experimental data of middle T')
        plt.plot(T, model_temp[10,:], label='Model prediction of middle T')
        plt.plot(T, exp_temp_end, label='Experimental data of end T')
        plt.plot(T, model_temp[19,:], label='Model prediction of end T')
        plt.xlabel('time [s]')
        plt.ylabel('Temperature [K]')
        plt.title('Temperature model prediction and experimental data')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()

## Gradient-based optimization 

In [11]:
# feed, bath, y
exp1 = generate_exp(293.15, 293.15, 0.5)



### Sequential initial solution 

In [13]:
sensi_opt = 'sequential_finite'

doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)
if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output=None,
                                formula='central')

result.calculate_FIM(doe_object.jac, doe_object.jac_3D, doe_object.design_values)


The flattend measurements include: ['FCO219', 'temp10', 'temp19']
Sensitivity information is scaled by its corresponding parameter nominal value.
This scenario: {'fitted_transport_coefficient': {0: 214.12}, 'ua': {0: 15.424948470398375}, 'jac-index': {'fitted_transport_coefficient': [0], 'ua': [0]}, 'eps-abs': {'fitted_transport_coefficient': 2.1412, 'ua': 0.15424948470398375}, 'scena-name': [0]}
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is 2021oct8_f313_b313_y15_5e6.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  293.15
yfeed is fixed at  0.5
Ipopt 3.13.3: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/c

 187r 0.0000000e+00 9.48e-01 1.57e+01  -1.2 3.47e-01  -1.1 1.00e+00 1.00e+00f  1
 188r 0.0000000e+00 1.35e-01 4.96e+00  -1.2 1.85e-01  -0.6 1.00e+00 1.00e+00h  1
 189r 0.0000000e+00 1.10e+00 1.78e+01  -1.2 4.05e-01  -1.1 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 4.12e-02 3.21e+00  -1.2 6.42e-01    -  1.00e+00 1.00e+00h  1
 191r 0.0000000e+00 6.32e-01 1.09e+02  -1.9 9.25e-01    -  7.57e-01 4.38e-01f  1
 192r 0.0000000e+00 6.59e-01 2.05e+02  -1.9 1.30e+00  -0.7 4.82e-01 2.50e-01f  1
 193r 0.0000000e+00 7.26e-01 1.63e+02  -1.9 2.68e+00  -1.2 8.16e-02 1.18e-01f  1
 194r 0.0000000e+00 7.21e-01 1.62e+02  -1.9 9.66e+00  -1.6 3.50e-02 2.88e-02f  1
 195r 0.0000000e+00 6.91e-01 1.47e+02  -1.9 3.13e+00  -1.2 7.58e-02 8.09e-02f  1
 196r 0.0000000e+00 6.80e-01 1.49e+02  -1.9 1.28e+01  -1.7 3.82e-02 2.63e-02f  1
 197r 0.0000000e+00 4.21e-01 8.69e+01  -1.9 5.51e-01  -0.4 3.16e-01 3.57e-01f  1
 198r 0.0000000e+00 4.89e-01

 370r 0.0000000e+00 1.50e+00 7.43e+01  -2.8 2.92e+00  -1.4 4.16e-02 8.63e-02f  1
 371r 0.0000000e+00 1.51e+00 7.55e+01  -2.8 1.02e+01  -1.9 1.70e-02 3.78e-02f  1
 372r 0.0000000e+00 1.51e+00 7.50e+01  -2.8 2.46e+01    -  7.78e-03 6.96e-03f  1
 373r 0.0000000e+00 1.49e+00 7.42e+01  -2.8 7.15e+00    -  2.12e-02 1.22e-02f  1
 374r 0.0000000e+00 1.48e+00 7.31e+01  -2.8 9.59e+00    -  2.24e-02 1.49e-02f  1
 375r 0.0000000e+00 1.50e+00 7.25e+01  -2.8 2.45e+01    -  1.58e-03 1.19e-02f  1
 376r 0.0000000e+00 1.49e+00 7.24e+01  -2.8 6.32e+00    -  2.09e-02 4.93e-03f  1
 377r 0.0000000e+00 1.54e+00 7.17e+01  -2.8 2.14e+01    -  1.37e-03 1.75e-02f  1
 378r 0.0000000e+00 1.54e+00 7.15e+01  -2.8 1.02e+01    -  1.22e-02 3.27e-03f  1
 379r 0.0000000e+00 1.59e+00 6.95e+01  -2.8 1.03e+01    -  1.33e-02 2.66e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380r 0.0000000e+00 1.63e+00 9.28e+01  -2.8 1.35e+01    -  9.00e-03 2.04e-02f  1
 381r 0.0000000e+00 1.66e+00


Number of Iterations....: 517

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
Constraint violation....:   9.7093842245287899e-09    1.4901161193847656e-08
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   9.7093842245287899e-09    1.4901161193847656e-08


Number of objective function evaluations             = 521
Number of objective gradient evaluations             = 5
Number of equality constraint evaluations            = 521
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 519
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 517
Total CPU secs in IPOPT (w/o function evaluations)   =    288.157
Total CPU secs in NLP function evalua

 180r 0.0000000e+00 3.03e+03 6.71e+03   1.7 9.32e-02   4.3 1.00e+00 1.00e+00f  1
 181r 0.0000000e+00 3.01e+03 1.61e+04   1.7 1.86e-01   3.8 1.00e+00 1.00e+00f  1
 182r 0.0000000e+00 3.03e+03 1.59e+04   1.7 6.76e-02   4.2 1.00e+00 1.00e+00f  1
 183r 0.0000000e+00 3.03e+03 3.45e+03   1.7 3.87e-02   4.7 1.00e+00 1.00e+00f  1
 184r 0.0000000e+00 3.03e+03 3.15e+03   1.7 7.55e-02   4.2 1.00e+00 1.00e+00f  1
 185r 0.0000000e+00 3.02e+03 8.71e+03   1.7 5.00e-01   3.7 4.69e-01 3.46e-01f  1
 186r 0.0000000e+00 3.02e+03 6.37e+03   1.7 1.31e-01   4.1 1.00e+00 7.41e-01f  1
 187r 0.0000000e+00 3.03e+03 3.13e+03   1.7 8.44e-02   4.6 1.00e+00 1.00e+00f  1
 188r 0.0000000e+00 3.02e+03 7.60e+04   1.7 1.74e-01   4.1 7.80e-01 7.67e-01f  1
 189r 0.0000000e+00 3.03e+03 3.35e+04   1.7 5.95e-02   4.5 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 3.01e+03 1.13e+04   1.7 1.08e-01   4.0 1.00e+00 1.00e+00f  1
 191r 0.0000000e+00 3.03e+03

...

 554r 0.0000000e+00 1.36e+00 2.94e+02  -4.2 4.26e+01  -2.1 1.24e-02 1.08e-03f  1
 555r 0.0000000e+00 1.36e+00 3.39e+02  -4.2 1.47e+01  -1.7 1.82e-02 1.01e-02f  1
 556r 0.0000000e+00 1.39e+00 3.30e+02  -4.2 3.00e+01  -2.2 3.21e-05 1.08e-02f  1
 557r 0.0000000e+00 1.39e+00 4.25e+02  -4.2 1.36e+01  -1.8 1.60e-02 1.81e-04f  1
 558r 0.0000000e+00 1.42e+00 4.31e+02  -4.2 3.78e+01  -2.3 3.06e-05 8.39e-03f  1
 559r 0.0000000e+00 1.42e+00 5.13e+02  -4.2 2.66e+01  -1.8 6.78e-03 8.80e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560r 0.0000000e+00 1.42e+00 7.88e+02  -4.2 1.56e-01   2.2 5.27e-02 1.53e-03h  1
 561r 0.0000000e+00 1.21e+00 6.02e+02  -4.2 1.56e-01   1.7 3.66e-02 1.45e-01f  1
 562r 0.0000000e+00 1.12e+00 5.53e+02  -4.2 1.33e-01   1.3 5.65e-02 7.75e-02h  1
 563r 0.0000000e+00 1.11e+00 5.47e+02  -4.2 1.73e-01   0.8 9.09e-02 5.13e-03h  1
 564r 0.0000000e+00 1.07e+00 5.25e+02  -4.2 2.84e-01   0.3 5.00e-02 3.63e-02h  1
 565r 0.0000000e+00 1.01e+00

The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is 2021oct8_f313_b313_y15_5e6.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  293.15
yfeed is fixed at  0.5
Ipopt 3.13.3: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.3, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:    93745
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    51003

Total number of variables.........................

 187r 0.0000000e+00 3.48e+00 1.15e+02  -1.9 7.20e+00  -1.4 4.97e-02 4.59e-02f  1
 188r 0.0000000e+00 3.09e+00 1.07e+02  -1.9 2.33e+00  -0.9 1.24e-01 9.50e-02f  1
 189r 0.0000000e+00 4.28e+00 9.81e+01  -1.9 1.32e+01  -1.4 2.68e-02 4.65e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 4.24e+00 9.90e+01  -1.9 3.46e+01  -1.9 1.73e-02 8.60e-03f  1
 191r 0.0000000e+00 4.02e+00 9.71e+01  -1.9 8.26e+00  -1.5 6.38e-02 4.42e-02f  1
 192r 0.0000000e+00 3.94e+00 9.28e+01  -1.9 6.99e+01  -1.9 3.03e-03 1.91e-02f  1
 193r 0.0000000e+00 3.86e+00 9.24e+01  -1.9 1.19e+01  -1.5 3.24e-02 1.82e-02f  1
 194r 0.0000000e+00 3.33e+00 7.77e+01  -1.9 3.29e+00  -1.1 5.37e-02 1.18e-01f  1
 195r 0.0000000e+00 3.31e+00 7.93e+01  -1.9 8.74e+00  -1.6 4.21e-02 7.42e-03f  1
 196r 0.0000000e+00 3.26e+00 7.76e+01  -1.9 5.33e+01  -2.0 3.75e-03 1.25e-02f  1
 197r 0.0000000e+00 3.20e+00 7.74e+01  -1.9 1.17e+01  -1.6 3.68e-02 1.73e-02f  1
 198r 0.0000000e+00 3.18e+00

 370r 0.0000000e+00 1.74e+00 6.00e+01  -2.9 4.54e+01    -  4.47e-03 7.06e-03f  1
 371r 0.0000000e+00 1.74e+00 5.95e+01  -2.9 2.86e+01    -  7.85e-03 6.67e-03f  1
 372r 0.0000000e+00 1.74e+00 5.93e+01  -2.9 4.25e+01    -  1.58e-03 6.50e-03f  1
 373r 0.0000000e+00 1.73e+00 5.85e+01  -2.9 7.67e+00    -  1.13e-02 3.09e-03f  1
 374r 0.0000000e+00 1.74e+00 5.84e+01  -2.9 1.95e+02    -  1.25e-04 1.53e-03f  1
 375r 0.0000000e+00 1.74e+00 5.82e+01  -2.9 3.08e+01    -  2.17e-03 1.18e-03f  1
 376r 0.0000000e+00 1.75e+00 5.86e+01  -2.9 3.51e+02    -  3.34e-04 6.37e-04f  1
 377r 0.0000000e+00 1.74e+00 5.79e+01  -2.9 9.13e+00    -  1.04e-02 3.60e-03f  1
 378r 0.0000000e+00 1.72e+00 5.73e+01  -2.9 1.80e+01    -  1.12e-02 1.38e-02f  1
 379r 0.0000000e+00 1.72e+00 5.71e+01  -2.9 7.53e+01    -  1.47e-03 2.17e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380r 0.0000000e+00 1.71e+00 1.00e+02  -2.9 9.79e+00  -1.6 8.66e-03 3.63e-02f  1
 381r 0.0000000e+00 1.76e+00


Number of Iterations....: 548

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
Constraint violation....:   7.4505805969238281e-09    1.4901161193847656e-08
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   7.4505805969238281e-09    1.4901161193847656e-08


Number of objective function evaluations             = 551
Number of objective gradient evaluations             = 5
Number of equality constraint evaluations            = 551
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 550
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 548
Total CPU secs in IPOPT (w/o function evaluations)   =    307.971
Total CPU secs in NLP function evalua

In [ ]:
print(doe_object.jac)

jac_init = np.zeros((len()))

for i, iname in enumerate(measurement):
    for j in parameter:
        for t in measurement_time:
            jac_init[i,j,t] = doe_object.jac[iname][]

In [17]:
fim_init = result.FIM

jac_init = doe_object.jac_3D.copy()
print(jac_init)
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

[[[ 0.00000000e+00  2.93557065e-06  5.24360317e-06  2.89373644e-05
    2.28597165e-04  1.69446909e-03  5.39116930e-03  3.36065306e-02
    8.54766910e-02  8.58004482e-02  7.26013397e-02  5.59373723e-02
    3.97192552e-02  2.54117339e-02  1.34367991e-02  3.76510872e-03
   -3.82007392e-03 -9.60021714e-03 -1.38637557e-02 -1.68796700e-02
   -1.88866992e-02 -2.00904357e-02 -2.06643216e-02 -2.07524537e-02
   -2.04730814e-02 -1.99222121e-02 -1.91770269e-02 -1.82989683e-02
   -1.73364520e-02 -1.63272023e-02 -1.53002338e-02 -1.42775162e-02
   -1.32753592e-02 -1.23055576e-02 -1.13763315e-02 -1.04930938e-02
   -9.65907415e-03 -8.87582254e-03 -8.14361448e-03 -7.46177484e-03
   -6.82893522e-03 -6.24323729e-03 -5.70249221e-03 -5.20430456e-03
   -4.74616780e-03 -4.32553681e-03 -3.93988227e-03 -3.58673053e-03
   -3.26369215e-03 -2.96848141e-03 -2.69892895e-03 -2.45298905e-03
   -2.22874280e-03 -2.02439821e-03 -1.83828809e-03 -1.66886627e-03
   -1.51470272e-03 -1.37447789e-03 -1.24697661e-03 -1.13108178

In [22]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         jac_involved_measurement = {'FCO2':[10], 'temp':[10,19]},
                                                         scale_nominal_param_value=True, objective_option='det', 
                                                         L_initial=l_init, jac_initial=jac_init, fim_initial=fim_init)

The flattend measurements include: ['FCO219', 'temp10', 'temp19']
Sensitivity information is scaled by its corresponding parameter nominal value.
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is 2021oct8_f293_b293_y50_5e6.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  293.15
yfeed is fixed at  0.5
Ipopt 3.13.3: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.3, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:   376912
Number of nonzeros in inequality 


Number of Iterations....: 156

                                   (scaled)                 (unscaled)
Objective...............:   2.6769441402093808e+00    2.6769441402093808e+00
Dual infeasibility......:   5.5203314097695966e-07    5.5203314097695966e-07
Constraint violation....:   1.6221083097610681e-11    1.8626451492309570e-08
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   1.6221083097610681e-11    5.5203314097695966e-07


Number of objective function evaluations             = 160
Number of objective gradient evaluations             = 5
Number of equality constraint evaluations            = 160
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 158
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 156
Total CPU secs in IPOPT (w/o function evaluations)   =    792.758
Total CPU secs in NLP function evalua

   7  2.6161383e+00 6.23e+01 1.04e+01  -1.0 8.79e+01    -  1.10e-03 1.67e-03h  1
   8  2.6161056e+00 6.23e+01 1.68e+02  -1.0 2.61e+02    -  1.70e-03 1.67e-05h  1
   9  2.6160797e+00 6.23e+01 2.33e+03  -1.0 2.94e+04    -  6.49e-06 1.25e-07h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  2.6150317e+00 6.23e+01 4.21e+03  -1.0 4.22e+04    -  6.24e-06 3.47e-06f  1
  11  2.6121925e+00 6.23e+01 7.51e+03  -1.0 4.32e+04    -  1.36e-05 9.08e-06f  1
  12  2.4799237e+00 1.28e+02 1.90e+05  -1.0 4.22e+04    -  2.56e-05 4.42e-04f  1
  13  2.4959726e+00 1.27e+02 1.92e+05  -1.0 1.02e+04    -  1.19e-04 1.83e-05f  1
  14  2.5421378e+00 1.20e+02 1.67e+05  -1.0 6.14e+03    -  1.10e-04 3.78e-05f  1
  15  2.6726029e+00 9.64e+01 1.00e+05  -1.0 5.35e+03    -  4.17e-04 1.60e-04f  1
  16  2.6674695e+00 9.62e+01 2.71e+05  -1.0 4.87e+03    -  5.94e-04 2.21e-05f  1
  17r 2.6674695e+00 9.62e+01 9.99e+02   1.8 0.00e+00    -  0.00e+00 1.09e-07R  2
  18r 2.6443990e+00 9.43e+01

  98  8.4291970e-01 4.11e-02 3.35e+08  -1.0 1.56e+00    -  1.00e+00 9.15e-03h  6
  99  8.5629562e-01 4.09e-02 1.11e+09  -1.0 1.54e+00    -  1.00e+00 9.12e-03h  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  8.6958119e-01 4.07e-02 3.68e+09  -1.0 1.52e+00    -  1.00e+00 9.10e-03h  6
 101  8.8277814e-01 4.05e-02 4.38e+09  -1.0 1.50e+00    -  1.00e+00 9.07e-03h  6
 102  9.0369803e-01 4.04e-02 1.37e+10  -1.0 1.48e+00    -  1.00e+00 1.44e-02h  5
 103  1.3633237e+00 1.75e-01 1.09e+12  -1.0 1.45e+00    -  1.00e+00 2.88e-01w  1
 104  1.3789056e+00 1.73e-01 1.52e+12  -1.0 9.67e-01    -  1.00e+00 1.29e-02w  1
 105  1.3790751e+00 1.73e-01 2.07e+12  -1.0 8.14e+00    -  1.96e-04 1.44e-04w  1
 106  9.2971893e-01 4.04e-02 3.57e+10  -1.0 4.90e+02    -  1.00e+00 1.80e-02h  4
 107  9.5539422e-01 4.03e-02 8.70e+10  -1.0 1.41e+00    -  1.00e+00 1.79e-02h  5
 108  9.8073438e-01 4.02e-02 1.34e+11  -1.0 1.38e+00    -  1.00e+00 1.78e-02h  5
 109  1.0057489e+00 4.01e-02

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190  2.6535715e+00 2.89e-04 5.59e+03  -1.0 1.20e-01    -  9.90e-01 5.67e-02h  2
 191  2.6586562e+00 2.68e-04 1.63e+04  -1.0 8.51e-03    -  2.65e-01 7.18e-02h  1
 192  2.6587190e+00 2.68e-04 1.30e+07  -1.0 4.01e-02    -  9.64e-01 9.72e-04h  1
 193r 2.6587190e+00 2.68e-04 1.00e+03  -1.0 0.00e+00    -  0.00e+00 2.68e-07R  8
 194r 2.3441106e+00 3.03e-02 1.64e+03  -1.0 4.55e-01    -  3.17e-01 2.02e-01f  1
 195r 1.3697386e+00 3.98e-01 2.11e+03  -1.0 4.02e-01    -  4.11e-01 7.29e-01f  1
 196r 1.1196949e+00 3.90e-02 2.61e+03  -1.0 1.81e-01   0.0 4.85e-01 1.00e+00f  1
 197r 7.9968740e-01 2.31e-01 9.50e+04  -1.0 2.12e-01    -  1.14e-01 1.00e+00f  1
 198r 8.0977342e-01 3.32e-02 2.39e+04  -1.0 5.84e-02   3.1 7.75e-01 1.00e+00f  1
 199r 8.0043233e-01 1.05e-02 1.86e+03  -1.0 8.56e-02    -  6.38e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200r 7.6203699e-01 5.26e-02

 282  2.6530916e+00 1.44e-03 8.30e+12  -1.0 2.40e-02    -  1.00e+00 6.91e-01H  1
 283  2.6889916e+00 7.19e-04 9.32e+13  -1.0 7.59e-03    -  4.38e-02 5.00e-01f  2
 284  2.7252497e+00 1.62e-05 2.22e+13  -1.0 3.59e-03    -  4.27e-01 1.00e+00h  1
 285  2.7251666e+00 2.24e-08 6.66e+10  -1.0 1.62e-05    -  9.44e-01 1.00e+00f  1
 286  2.7251666e+00 1.86e-08 1.64e+08  -1.0 2.73e-08    -  9.90e-01 1.00e+00h  1
 287  2.7251667e+00 1.86e-08 1.63e+06  -1.0 2.73e-06    -  9.90e-01 1.00e+00f  1
 288  2.7251759e+00 5.44e-08 1.22e+04  -1.0 2.68e-04    -  9.93e-01 1.00e+00f  1
 289  2.7253896e+00 3.42e-05 1.34e-02  -1.0 9.51e-03    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290  2.7253906e+00 1.86e-08 1.70e-02  -2.5 8.48e-05    -  1.00e+00 1.00e+00h  1
 291  2.7253818e+00 1.19e-06 2.83e-03  -3.8 2.71e-04    -  1.00e+00 1.00e+00h  1
 292  2.7253028e+00 4.20e-06 3.03e-04  -3.8 2.63e-03    -  1.00e+00 1.00e+00h  1
 293  2.7252325e+00 3.81e-06

In [ ]:
jac_init_1 = doe_object.analysis_square.jac_extracted.copy()
print(type(jac_init_1))
print(jac_init_1[0,0,0])

print(jac_init_1)

fo = open('jac_initial', 'wb')
pickle.dump(jac_init_1, fo)
fo.close



In [23]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

======Result summary======
This optimization is solved with status: converged
It gives solution: {'temp_feed': [273.15], 'yfeed': [0.5]}
The log10(OF) optimized is: -2.725156736668212
The result FIM is: [[0.03910728 0.0202383 ]
 [0.0202383  1.68627229]]


AttributeError: 'ConcreteModel' object has no attribute 'T'

## Heatmap

In [ ]:
#design_ranges = [list(np.linspace(293,333,5)), list(np.linspace(0.1,0.3,6))]
design_ranges = [[293, 303, 313, 323, 333], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

dv_apply_name = ['temp_feed','yfeed']

dv_apply_time = [[0],[0]]

# Define experiments
exp1 = generate_exp(313.15, 303.15, 0.15)
    
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

print(design_ranges)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, tee_option=False,
                                    scale_nominal_param_value=True, 
                                     store_name='20210919_heatmap', read_name='20210919_heatmap'
                                    )



In [ ]:
test = all_fim.extract_criteria()

fixed = {}
all_fim.figure_drawing(fixed, ['temp_feed','yfeed'], 'Fixed bed','$feed T [K]$', 'y' )

In [ ]:
na = ['temp_feed', 'yfeed']
dd = ['temp_feed', 'yfeed']
a = [[293.0, 303.0, 313.0, 323.0, 333.0], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

In [ ]:
print(a[na.index('temp_feed')])